In [3]:
import os
import numpy as np
import pandas as pd 
from PIL import Image

from fastai import *
from fastai.vision.all import *
from fastai.callback import *

#### This notebook is the inference of model trained in [here](https://www.kaggle.com/mabelwang21/plant-pathology-fastai-lu-train/edit). Now we can turn off the internet, load the trained model, and do inference.

In [5]:
path = Path("../input")
data_path = path/'plant-pathology-2021-fgvc8'

In [6]:
def get_x(r):  return data_path/"train_images"/r["image"]
def get_y(r):  return r["labels"]
f1_score_multi = F1Score(average="weighted")

In [7]:
learn = load_learner('../input/baselinemodel/resnet50_baseline.pkl', cpu=False)

In [8]:
 learn.to_fp32()

In [11]:
test = get_image_files(data_path/'test_images')
test_dl = learn.dls.test_dl(test)
preds = learn.get_preds(dl=test_dl) # get prediction
preds_idx =  np.argmax(to_np(preds[0]),1)
preds_idx

array([0, 1, 9])

In [12]:
learn.dls.vocab

['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot complex', 'healthy', 'powdery_mildew', 'powdery_mildew complex', 'rust', 'rust complex', 'rust frog_eye_leaf_spot', 'scab', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex']

In [13]:
to_submit = pd.DataFrame({'image':os.listdir(data_path/'test_images')})
to_submit['labels'] = learn.dls.vocab[preds_idx]
to_submit.to_csv("submission.csv",index=False)